In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import subprocess
from scipy.optimize import minimize
import os
import shutil
import math
from fredapi import Fred
from statsmodels.tsa.filters.hp_filter import hpfilter
import statsmodels.api as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.stattools import adfuller



In [ ]:
#Generates Empirical Moments from FRED Data--------

#Enter FRED API KEY
fred=Fred(api_key="")

#Enter Country specific series name from FRED in the order - Total GDP, Private Final Consumption Expenditure, Gross Fixed Capital Fromation, Exports, Imports by Expenditure Method at Constant Prices
series=["NAEXKP01INQ652S","NAEXKP02INQ189S","NAEXKP04INQ652S","NAEXKP06INQ652S","NAEXKP07INQ652S","AUSGDPRQDSMEI","NAEXKP02AUQ189S","NAEXKP04AUQ189S","NAEXKP06AUQ652S","NAEXKP07AUQ652S","NAEXKP01ZAQ652S","NAEXKP02ZAQ189S","NAEXKP04ZAQ652S","NAEXKP06ZAQ652S","NAEXKP07ZAQ652S","NAEXKP01BRQ652S","NAEXKP02BRQ189S","NAEXKP04BRQ652S","NAEXKP06BRQ652S","NAEXKP07BRQ652S","NAEXKP01BEQ189S","NAEXKP02BEQ189S","NAEXKP04BEQ189S","NAEXKP06BEQ652S","NAEXKP07BEQ652S","NAEXKP01DKQ189S","NAEXKP02DKQ189S","NAEXKP04DKQ189S","NAEXKP06DKQ652S","NAEXKP07DKQ652S","NAEXKP01NZQ189S","NAEXKP02NZQ189S","NAEXKP04NZQ189S","NAEXKP06NZQ652S","NAEXKP07NZQ652S","NAEXKP01CAQ189S","NAEXKP02CAQ189S","NAEXKP04CAQ189S","NAEXKP06CAQ652S","NAEXKP07CAQ652S","NAEXKP01CHQ652S","NAEXKP02CHQ189S","NAEXKP04CHQ652S","NAEXKP06CHQ652S","NAEXKP07CHQ652S","NAEXKP01MXQ189S","NAEXKP02MXQ189S","NAEXKP04MXQ189S","NAEXKP06MXQ652S","NAEXKP07MXQ652S"]
countries=["In","Aus","SA","Brz","Bel","Den","NZ","Can","Swi","Mex"]    # Add Country Name
variables=["GDP","Cons","Inv","X","M"]      # Do Not Modify
c_v=[i+"_"+var for i in countries for var in variables]
var_names={}
for index,i in enumerate(c_v):
    var_names[i]=(fred.get_series(series[index],observation_start="1996-04-01",observation_end="2022-10-01"))      #Change Start and End Date based on requirement, and ensure countries have data in that period, else missing values will cause NA estimates
for i in var_names:
    var_names[i]=var_names[i].reset_index()
    var_names[i].columns=["Date",i]
for i in countries:                                 #Calculating Net Exports
    nx_var=i+"_NX"
    var_names[nx_var]=pd.DataFrame()
    var_names[nx_var][nx_var]=var_names[i+"_X"][i+"_X"]-var_names[i+"_M"][i+"_M"]
    var_names[nx_var]["Date"]=var_names[i+"_GDP"]["Date"]
for i in var_names:                                  #Dropping Missing values if Any
    var_names[i]=var_names[i].dropna()
for i in countries:                                 #Calculating NX/Y
    nx_var=i+"_NX"
    nxy_var=i+"_NX/Y"
    var_names[nxy_var]=pd.DataFrame()
    var_names[nxy_var][nxy_var]=var_names[nx_var][nx_var]/var_names[i+"_GDP"][i+"_GDP"]
    var_names[nxy_var]["Date"]=var_names[i+"_GDP"]["Date"]
for i in var_names:                                 #Taking Logs of Variables
    if "NX" not in i:
        var_names[i][i]=np.log(var_names[i][i])

#Detrending with HP filter
for i in var_names:
    cycle,trend = hpfilter(var_names[i][i],lamb=1600)
    detrended_var=i+"_"+"cycle"
    var_names[i][detrended_var]=cycle

#Find Correlation and Volatility - Calculating Moments
stats={}
for i in var_names:
    if "Cons" in i:
        cons_var=i+"_Var"
        stats[cons_var]=var_names[i][i+"_cycle"].std()*100
    if "GDP" in i:
        gdp_var=i+"_Var"
        stats[gdp_var]=var_names[i][i+"_cycle"].std()*100
    if "Inv" in i:
        inv_var=i+"_Var"
        stats[inv_var]=var_names[i][i+"_cycle"].std()*100
    if "NX/Y" in i:
        nxy_var=i+"_Var"
        stats[gdp_var]=nxy_names[i][i+"_cycle"].std()*100
temp={}
for i in stats:
    if "GDP" in i:
        gdp_temp=stats[i]
    if "Cons" in i:
        name=i.split("_")[0]
        temp[name+"_C/Y"]=stats[i]/gdp_temp
    if "Inv" in i:
        name=i.split("_")[0]
        temp[name+"_I/Y"]=stats[i]/gdp_temp
    if "NX/Y" in i:
        name=i.split("_")[0]
        temp[name+"_NXY/Y"]=stats[i]/gdp_temp
stats.update(temp)
del temp, gdp_temp, cons_var, gdp_var
for i in countries:
    corr_var=i+"_CorrNXY"
    stats[corr_var]=var_names[i+"_NX/Y"][i+"_NX/Y_cycle"].corr(var_names[i+"_GDP"][i+"_GDP_cycle"])
    corr_var=i+"_CorrC"
    stats[corr_var]=var_names[i+"_Cons"][i+"_Cons_cycle"].corr(var_names[i+"_GDP"][i+"_GDP_cycle"])
    corr_var=i+"_CorrI"
    stats[corr_var]=var_names[i+"_Inv"][i+"_Inv_cycle"].corr(var_names[i+"_GDP"][i+"_GDP_cycle"])


In 0.9239726657931175
Aus 0.5362359406262245
SA 0.7012179534147143
Brz 0.843654443191396
Bel 0.6772474828403319
Den 0.602638581825615
NZ 0.6797821974950844
Can 0.6687140656067526
Swi 0.7197463041274023
Mex 0.9274989100747032


In [ ]:
#Calculate Empirical Moments for Country
def calculate_moments(country):
    emp_moments=[[stats[f"{country}_GDP_Var"]],[stats[f"{country}_C/Y"]],[stats[f"{country}_CorrC"]]]
    return emp_moments

emp_moments=calculate_moments("Swi")    #Outputs Empirical Moments for Country given as argument, Note - Country must be refered to as defined earlier

In [ ]:
# Save parameters to a .mat file for Dynare to load
filepath2="" #Filepath to folder where aguiar_gopinath_soe.mod and param.m are saved
def save_params_to_m(params, filename=filepath2+"param2.m"):
    sigma_g,sigma_z,rho_g= params  # Parameters to be estimated

    with open(filename, "w") as f:
        # Fixed parameters from your model
        f.write("// Benchmark parameter values (Table 3)\n")
        f.write("beta = 1/1.02;\n")
        f.write("gamma = 0.36;\n")
        f.write("b_share = 0.1; \n")
        f.write("psi = 0.001;\n")
        f.write("alpha = 0.68;\n")
        f.write("b_star = 0.1;\n")
        f.write("r_star = 0.05;\n")
        f.write("sigma = 2;\n")
        f.write("delta = 0.05;\n")
        # Updated parameters
        phi=4
        mu_g=math.log(1.0067)
        rho_z=0.95
        f.write(f"sigma_g = {sigma_g};\n")
        f.write(f"sigma_z = {sigma_z};\n")
        f.write(f"mu_g = {mu_g};\n")
        f.write(f"phi = {phi}; // updated\n")
        f.write(f"rho_z = {rho_z}; // updated\n")
        f.write(f"rho_g = {rho_g}; // updated\n")
     

#Run Dynare
def run_dynare(dynare_dir=filepath2):
    global modname  # Declare as global
    modfile = "aguiar_gopinath_soe.mod"                                 #Filepath to dynare file
    modname=modfile.split(".mod")[0]
    mat_path = filepath2+f"{modname}/Output/{modname}_results.mat"   #Filepath to outputted mat file - Not to be modified 
    results_folder = filepath2+modname

    # Clean up previous results
    if os.path.exists(results_folder):
        shutil.rmtree(results_folder)

    # Run Dynare
    command = f"dynare {modfile} nolog"                 #Ensure octave, and thus can be run from the terminal
    subprocess.run(["octave", "--eval", command],       #Runs Octave as a subprocess 
                   cwd=dynare_dir,
                   text=True,
                   stdout=subprocess.DEVNULL,
                   stderr=subprocess.DEVNULL,
                   )

    # 🔧 Check if .mat was created
    return os.path.exists(mat_path)                     #Returns True if the code ran sucessfully, and outputted a .mat file esle False

#Load the simulation data outputed by dynare
def load_model_data():
    filepath=filepath2=f"{modname}/Output/{modname}_results.mat"
    data = scipy.io.loadmat(filepath,simplify_cells=True)
    oo_ = data['oo_']
    M_ = data['M_']
    sim_mom = oo_['simulated_moments']  # shape: (n_vars, T)
    y_std=sim_mom["standard_devs"][0]
    c_std=sim_mom["standard_devs"][1]
    dy_std=sim_mom["standard_dev_first_diff"]
    cy_std=sim_mom["relative_standard_devs"][1]
    iy_std=sim_mom["relative_standard_devs"][2]
    nxy_std=sim_mom["relative_standard_devs"][3]
    rho_y=sim_mom["autocorrelations"][0]
    rho_dy=sim_mom["autocorrelations"][1]
    rho_c=sim_mom["correlations"][0]
    rho_i=sim_mom["correlations"][1]
    rho_nx=sim_mom["correlations"][2]
    #[[y_std],[cy_std],[rho_c],[rho_nx],[dy_std],[iy_std],[rho_y],[rho_dy],[nxy_std],[rho_i]]   
    sim=[[y_std],[cy_std],[rho_c]]     #Creating 2*1 Vector
    return np.asarray(sim)


#GMM Loss func
def gmm_loss(params,emp_moments):
    #Weight Matrix I
    W=np.eye(3)                 #3 Moments being used, thus a 3*3 Identity Matix
    print("The parameters are", params)
    save_params_to_m(params)  # Step 1: Save
    print("ParametersSaved")
    success = run_dynare()  # Step 2: Run Dynare
    if not success:
        print("Dynare failed to generate .mat file (likely BK condition failed)")
        return 1e10  # If Model does not converge, penalize loss function with high loss

    print("DynareRun")

    try:
        model_moments = load_model_data()  # Step 3: Load results
        error=(model_moments-emp_moments)             #Creates a 3*1 error vector
        loss=error.T @ W @ error
        print(f"Loss: {loss}")
        return loss
    except Exception as e:
        print(f"Error reading model output or computing moments: {e}")
        return 1e10  # 🔧 Penalize on exception too

initial_guess = [1,1,1]

# Run GMM estimation
result = minimize(gmm_loss, initial_guess, args=(emp_moments,), bounds=((1e-2,None),(1e-2,None),(0,None)),method="L-BFGS-B",options={"gtol":1e-6})

# Print final result
print("\n Estimated Parameters:")
print(result.x)

The parameters are [1. 1. 1.]
ParametersSaved
DynareRun
Loss: [[365.0798217]]
The parameters are [1.00000001 1.         1.        ]
ParametersSaved
DynareRun
Loss: [[365.07982942]]
The parameters are [1.         1.00000001 1.        ]
ParametersSaved
DynareRun
Loss: [[365.07982175]]
The parameters are [1.         1.         1.00000001]
ParametersSaved
DynareRun
Loss: [[365.08001035]]
The parameters are [0.42459363 0.42459363 0.41878144]
ParametersSaved
DynareRun
Loss: [[0.23415087]]
The parameters are [0.42459364 0.42459363 0.41878144]
ParametersSaved
DynareRun
Loss: [[0.23415087]]
The parameters are [0.42459363 0.42459364 0.41878144]
ParametersSaved
DynareRun
Loss: [[0.23415086]]
The parameters are [0.42459363 0.42459363 0.41878145]
ParametersSaved
DynareRun
Loss: [[0.23415087]]
The parameters are [0.42474405 0.42477554 0.41885541]
ParametersSaved
DynareRun
Loss: [[0.23375265]]
The parameters are [0.42474406 0.42477554 0.41885541]
ParametersSaved
DynareRun
Loss: [[0.23375264]]
The par

AttributeError: summary